In [1]:
# %matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import signal
import matplotlib.animation as animation
import os
import pickle

'''set parameter'''

samplerate = 5              # サンプリングレート[fps]
fp = 0.2                      # 通過域端周波数[Hz] 2
fs = 2                      # 阻止域端周波数[Hz] 3
gpass = 1                   # 通過域最大損失量[dB] 1
gstop = 2                   # 阻止域最小減衰量[dB] 40

'''---------------------------'''

# butterworth low-pass filter
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y        

# get data
data = []
# number = 1 # 1 to 22
for number in range(1,23):
#     print(number)
    name = 'peregrine_' +  "%02d" % number + '.xlsx'
    filename = './' + name
    xlsx_data = pd.read_excel(filename, encoding="utf_8", sep=",")
    xlsx_data = np.array(xlsx_data)

    
    # print(40000 / 360)
    # 1 (deg) : 111111 (m) = data (deg) : x (m) 
    # x (m) = 111111 * data: 

    # pp = Peregrine Position
    pp1 = xlsx_data[:,0] * 111111
    pp2 = xlsx_data[:,1] * 111111
    pp3 = xlsx_data[:,2]

    # tp = Target Position
    tp1 = xlsx_data[:,4] * 111111
    tp2 = xlsx_data[:,5] * 111111
    tp3 = xlsx_data[:,6]

    # low-pass filter
    pp1 = lowpass(pp1, samplerate, fp, fs, gpass, gstop)
    pp2 = lowpass(pp2, samplerate, fp, fs, gpass, gstop)
    pp3 = lowpass(pp3, samplerate, fp, fs, gpass, gstop)
    tp1 = lowpass(tp1, samplerate, fp, fs, gpass, gstop)
    tp2 = lowpass(tp2, samplerate, fp, fs, gpass, gstop)
    tp3 = lowpass(tp3, samplerate, fp, fs, gpass, gstop)
    
    # shift target position
    pp1_end = pp1[-1]
    pp2_end = pp2[-1]
    pp3_end = pp3[-1]
    
    tp1_end = tp1[-1]
    tp2_end = tp2[-1]
    tp3_end = tp3[-1]
    
    dis1_end = pp1_end - tp1_end
    dis2_end = pp2_end - tp2_end
    dis3_end = pp3_end - tp3_end
    
    tp1 += dis1_end 
    tp2 += dis2_end
    tp3 += dis3_end
    
    # shift to origin
    x_min = np.min([np.min(pp1), np.min(tp1)])
    x_max = np.max([np.max(pp1), np.max(tp1)])
    y_min = np.min([np.min(pp2), np.min(tp2)])
    y_max = np.max([np.max(pp2), np.max(tp2)])
    z_min = np.min([np.min(pp3), np.min(tp3)])
    z_max = np.max([np.max(pp3), np.max(tp3)]) 

    pp1 -= x_min
    pp2 -= y_min
    pp3 -= z_min
    tp1 -= x_min 
    tp2 -= y_min
    tp3 -= z_min

    # calculate velocity
    dpp1 = np.diff(pp1)
    dpp2 = np.diff(pp2)
    dpp3 = np.diff(pp3)
    dtp1 = np.diff(tp1)
    dtp2 = np.diff(tp2)
    dtp3 = np.diff(tp3)

    # ajust position data length to velocity data
    pp1 = pp1[:-1]
    pp2 = pp2[:-1]
    pp3 = pp3[:-1]
    tp1 = tp1[:-1]
    tp2 = tp2[:-1]
    tp3 = tp3[:-1]
    
    print('pp1', pp1)
    print('pp2', pp2)
    print('pp3', pp3)

    # get speed from the original data file
    ps = xlsx_data[:-1,3]
    ts = xlsx_data[:-1,7]

    p_data = []
    p_data.append(dpp1); p_data.append(dpp2); p_data.append(dpp3)
    p_data.append(pp1); p_data.append(pp2); p_data.append(pp3)
    p_data.append(ps)
    
    t_data = []
    t_data.append(dtp1); t_data.append(dtp2); t_data.append(dtp3)
    t_data.append(tp1); t_data.append(tp2); t_data.append(tp3)
    t_data.append(ts)
    
    data_seq = []
    data_seq.append(p_data)
    data_seq.append(t_data)
    
    data.append(data_seq)
    
print(np.array(p_data).shape, np.array(t_data).shape, np.array(data_seq).shape, np.array(data).shape)
    
np.save('peregrine_data', data)
# print(np.load('data.npy'))
   

pp1 [98.35982646699995 99.17987706698477 100.0630463194102 101.0424108793959
 102.14407781139016 103.39008094277233 104.78674902021885
 106.31614364217967 107.94388458225876 109.62637733947486
 111.31124079227448 112.9447899479419 114.48177430126816 115.8814804116264
 117.0929990047589 118.03973267506808 118.6601383946836 118.96452590078115
 118.99913672916591 118.80788448546082 118.44011211208999
 117.94462444260716 117.35397485643625 116.7087022587657
 116.06109992414713 115.42287640832365 114.78028126806021
 114.12667710799724 113.45380394626409 112.74742681812495
 111.98367599677294 111.13096416834742 110.15502117853612
 109.03188694175333 107.76431383285671 106.37054257374257
 104.86903912667185 103.27927396632731 101.61818926688284
 99.89746146183461 98.12424093205482 96.30176808685064 94.43042909167707
 92.51082772389054 90.54700198117644 88.54616402834654 86.52188690192997
 84.49290489219129 82.4786136886105 80.49425531271845 78.53902715351433
 76.59534561354667 74.639224168844

pp1 [13.659311588853598 13.658516179770231 13.657917839474976
 13.65749802812934 13.65715374238789 13.656765334308147 13.656230754218996
 13.655562810599804 13.65481955371797 13.654083035886288 13.65345897525549
 13.653002355247736 13.652695380151272 13.652464696206152
 13.652198603376746 13.651790700852871 13.651155708357692
 13.650155982002616 13.648725911974907 13.64700793940574
 13.645185638219118 13.643407871946692 13.641769520938396
 13.640256129205227 13.638791195116937 13.637259860523045
 13.635580909438431 13.63375841267407 13.631795218214393 13.62969369534403
 13.627587731927633 13.625690329819918 13.624072234146297
 13.622651329264045 13.621338920667768 13.620060225017369
 13.61874931678176 13.617455877363682 13.616314459592104
 13.615346292033792 13.614486754871905 13.613765454851091
 13.613219127058983 13.612718826159835 13.612118506804109
 13.611428302712739 13.610787996090949 13.6103212852031 13.609963932074606
 13.609549064189196 13.608938487246633 13.60803016461432
 13

In [2]:
import numpy as np

data = np.load('/Users/tsutsuikazushi/Documents/Nagoya/animals/peregrine_data_meter.npy', allow_pickle=True)

print(np.array(data).shape)



(22, 2, 7, 100)


In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import matplotlib.animation as animation
import os

'''set parameter'''
samplerate = 1              # サンプリングレート[fps]
fp = 0.02                      # 通過域端周波数[Hz] 2
fs = 0.3                      # 阻止域端周波数[Hz] 3
gpass = 1                   # 通過域最大損失量[dB] 1
gstop = 2                   # 阻止域最小減衰量[dB] 40

'''---------------------------'''

# butterworth low-pass filter
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y        


# get data
data = []
# number = 1 # 1 to 6
for number in range(1, 7):
    name = 'sula_' + "%02d" % number + '.xls'
    filename = '/Users/tsutsuikazushi/Documents/Nagoya/animals/data/sula/' + name

    dat_original = pd.read_excel(filename, header=1)
    
    
    if number == 1 or number == 2 or number == 6:
        dat = dat_original[['long1', 'lat1', 'speed1', 'long2', 'lat2', 'speed2']]
        dat = dat.interpolate()

        bird_a1 = np.array(dat['long1']) * 111111
        bird_a2 = np.array(dat['lat1']) * 111111
        bird_b1 = np.array(dat['long2']) * 111111
        bird_b2 = np.array(dat['lat2']) * 111111

        # low-pass filter
        bird_a1 = lowpass(bird_a1, samplerate, fp, fs, gpass, gstop)
        bird_a2 = lowpass(bird_a2, samplerate, fp, fs, gpass, gstop)
        bird_b1 = lowpass(bird_b1, samplerate, fp, fs, gpass, gstop)
        bird_b2 = lowpass(bird_b2, samplerate, fp, fs, gpass, gstop)
        
        # shift to origin
        x_min = np.min([np.min(bird_a1), np.min(bird_b1)])
        x_max = np.max([np.max(bird_a1), np.max(bird_b1)])
        y_min = np.min([np.min(bird_a2), np.min(bird_b2)])
        y_max = np.max([np.max(bird_a2), np.max(bird_b2)])

        bird_a1 -= x_min 
        bird_a2 -= y_min
        bird_b1 -= x_min 
        bird_b2 -= y_min

        # calculate velocity
        dba1 = np.diff(bird_a1)
        dba2 = np.diff(bird_a2)
        dbb1 = np.diff(bird_b1)
        dbb2 = np.diff(bird_b2)

        # ajust position data length to velocity data
        bird_a1 = bird_a1[:-1]
        bird_a2 = bird_a2[:-1]
        bird_b1 = bird_b1[:-1]
        bird_b2 = bird_b2[:-1]

        # get speed from the original data file
        bird_as = np.array(dat['speed1'])
        bird_bs = np.array(dat['speed2'])

        a_data = []
        a_data.append(dba1); a_data.append(dba2); a_data.append(bird_a1); a_data.append(bird_a2);
        a_data.append(bird_as)

        b_data = []
        b_data.append(dbb1); b_data.append(dbb2); b_data.append(bird_b1); b_data.append(bird_b2);
        b_data.append(bird_bs)
        
        data_seq = []
        data_seq.append(a_data)
        data_seq.append(b_data)
        
        data.append(data_seq)
        
        
    elif number == 3 or number == 4 or number == 5:
        dat = dat_original[['long1', 'lat1', 'speed1', 'long2', 'lat2', 'speed2', 'long3', 'lat3', 'speed3']]
        dat = dat.interpolate()

        bird_a1 = np.array(dat['long1']) * 111111
        bird_a2 = np.array(dat['lat1']) * 111111
        bird_b1 = np.array(dat['long2']) * 111111
        bird_b2 = np.array(dat['lat2']) * 111111
        bird_c1 = np.array(dat['long3'])* 111111
        bird_c2 = np.array(dat['lat3']) * 111111
        
        # low-pass filter
        bird_a1 = lowpass(bird_a1, samplerate, fp, fs, gpass, gstop)
        bird_a2 = lowpass(bird_a2, samplerate, fp, fs, gpass, gstop)
        bird_b1 = lowpass(bird_b1, samplerate, fp, fs, gpass, gstop)
        bird_b2 = lowpass(bird_b2, samplerate, fp, fs, gpass, gstop)
        bird_c1 = lowpass(bird_c1, samplerate, fp, fs, gpass, gstop)
        bird_c2 = lowpass(bird_c2, samplerate, fp, fs, gpass, gstop)
        
        # shift to origin
        x_min = np.min([np.min(bird_a1), np.min(bird_b1), np.min(bird_c1)])
        x_max = np.max([np.max(bird_a1), np.max(bird_b1), np.max(bird_c1)])
        y_min = np.min([np.min(bird_a2), np.min(bird_b2), np.min(bird_c2)])
        y_max = np.max([np.max(bird_a2), np.max(bird_b2), np.max(bird_c2)])

        bird_a1 -= x_min
        bird_a2 -= y_min
        bird_b1 -= x_min 
        bird_b2 -= y_min
        bird_c1 -= x_min
        bird_c2 -= y_min
        
        # calculate velocity
        dba1 = np.diff(bird_a1)
        dba2 = np.diff(bird_a2)
        dbb1 = np.diff(bird_b1)
        dbb2 = np.diff(bird_b2)
        dbc1 = np.diff(bird_c1)
        dbc2 = np.diff(bird_c2)
        
        # ajust position data length to velocity data
        bird_a1 = bird_a1[:-1]
        bird_a2 = bird_a2[:-1]
        bird_b1 = bird_b1[:-1]
        bird_b2 = bird_b2[:-1]
        bird_c1 = bird_c1[:-1]
        bird_c2 = bird_c2[:-1]
        
        # get speed from the original data file
        bird_as = np.array(dat['speed1'])
        bird_bs = np.array(dat['speed2'])
        bird_cs = np.array(dat['speed3'])

        a_data = []
        a_data.append(dba1); a_data.append(dba2); a_data.append(bird_a1); a_data.append(bird_a2);
        a_data.append(bird_as)

        b_data = []
        b_data.append(dbb1); b_data.append(dbb2); b_data.append(bird_b1); b_data.append(bird_b2);
        b_data.append(bird_bs)
        
        c_data = []
        c_data.append(dbc1); c_data.append(dbc2); c_data.append(bird_c1); c_data.append(bird_c2);
        c_data.append(bird_cs)
        
        data_seq = []
        data_seq.append(a_data)
        data_seq.append(b_data)
        data_seq.append(c_data)
        
        data.append(data_seq)
        
np.save('/Users/tsutsuikazushi/Documents/Nagoya/animals/sula_data_meter', data)
       

In [2]:
import numpy as np

data = np.load('/Users/tsutsuikazushi/Documents/Nagoya/animals/sula_data_meter.npy', allow_pickle=True)

print(np.array(data[0][1][1]).shape)



(43226,)
